# Capstone assignment - Toronto neighborhood clustering

In [47]:
from bs4 import BeautifulSoup

In [48]:
import requests

In [49]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [50]:
soup = BeautifulSoup(source, 'lxml')

In [51]:
#print(soup.prettify())   

In [52]:
table = soup.find("table",{"class":"wikitable sortable"})

Function to parses a html segment started with tag table followed by multiple 'tr' (table rows) and inner 'td' (table data) tags. It returns a list of rows with inner columns.  Accepts only one 'th' (table header/data) in the first row.
   

In [53]:
def tableDataText(table):    
    
    def rowgetDataText(tr, coltag='td'): # td (data) or th (header)       
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]  
    rows = []
    trs = table.find_all('tr')
    headerow = rowgetDataText(trs[0], 'th')
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append(rowgetDataText(tr, 'td') ) # data row       
    return rows

In [54]:
list_table = tableDataText(table)

In [55]:
import pandas as pd

dftable = pd.DataFrame(list_table[1:], columns=list_table[0])

Drop the "Not assigned" rows

In [56]:
dftable_notassigned = dftable[dftable['Borough']=='Not assigned']

dftable = dftable.drop(dftable_notassigned.index, axis=0)


In [57]:
dftable.shape

(103, 3)

In [58]:
geodata = pd.read_csv("http://cocl.us/Geospatial_data")


Convent the column name, so that the two data frame have identical column 'Postal code'
Merge the two data frames on 'Postal code'

In [59]:
geodata.rename(columns={'Postal Code':'Postal code'}, inplace=True)
geodata.columns

Index(['Postal code', 'Latitude', 'Longitude'], dtype='object')

In [60]:
neighbourhoods=pd.merge(dftable, geodata, on='Postal code')


In [63]:
# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt

Libraries imported.


In [64]:
latitude=43.662696
longitude=-79.400049
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto